In [90]:
#Import necessary Python dependencies #From Starter Code

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import datetime as dt

In [91]:
#Import necessary SqlAlch dependencies that include tool kit and ORM #From Starter Code

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [92]:
#Use create_engine link engine with data base or in this the sqlite file we will use for this portion of the hw
# From Starter Code 

engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [93]:
#Reflect existing db and tbale into a new model/object
#From starter code 

Base = automap_base()

Base.prepare(engine, reflect=True)

In [94]:
#Print out classes that exists in the model above
#From Starter Code
Base.classes.keys()

['measurement', 'station']

In [95]:
#Save classes as references stored in variables 
#From Starter Code 
Station = Base.classes.station
Measurement = Base.classes.measurement

In [96]:
#Create session to activate engine that links Python to DB
#From Starter Code 
session = Session(engine)

In [97]:
#Query to retrieve the last 12 months of precipitation data print total number of results
engine.execute("select * from Measurement limit 12").fetchall()
session.query(func.count(Measurement.date)).all()

[(19550)]

In [98]:
#Prior to calculating the date 1 year ago from last date, we need to find the last date. 
#An alternative way to getting this result would have to completed an order_by on Measurement.date and then list out the first object in descending order. I just find this method easier

LastDate = session.query(func.max(Measurement.date)).all()

LastDate

[('2017-08-23')]

In [99]:
#The only information I need is the string value from within the queried object above. 
#Use of np.ravel function is to extract the string. We learned this function in class. 
LastDate = list(np.ravel(LastDate))[0]

LastDate


'2017-08-23'

In [100]:
#Prior to calculating the year ago date, I need to convert this string into a date.time object
LastDate = dt.datetime.strptime(LastDate, "%Y-%m-%d")

LastDate

datetime.datetime(2017, 8, 23, 0, 0)

In [106]:
#Because we only need the first three values in our object for the date, we need to extract each of those values as unique elements of the date.
#Additionally I wrapped each of these values as integers because that will help me when I query the necessary prcp data

LastMonth = int(dt.datetime.strftime(LastDate, "%m"))
LastDay = int(dt.datetime.strftime(LastDate, "%d"))
LastYear = int(dt.datetime.strftime(LastDate, "%Y"))

print(f"{LastMonth}-{LastDay}-{LastYear}")


8-23-2017


In [107]:
#Calculation of 'year-ago' date from the lastdate 

year_ago_date = dt.date(LastYear, LastMonth, LastDay) - dt.timedelta(days=365)
year_ago_date

datetime.date(2016, 8, 23)

In [114]:
#Query to retrieve the data and precipitation values within this specific year frame

LastYear_PrcpData = (session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > year_ago_date).order_by(Measurement.date).all())
LastYear_PrcpData

[('2016-08-24', 0.08),
 ('2016-08-24', 2.15),
 ('2016-08-24', 2.28),
 ('2016-08-24', None),
 ('2016-08-24', 1.22),
 ('2016-08-24', 2.15),
 ('2016-08-24', 1.45),
 ('2016-08-25', 0.08),
 ('2016-08-25', 0.08),
 ('2016-08-25', 0.0),
 ('2016-08-25', 0.0),
 ('2016-08-25', 0.21),
 ('2016-08-25', 0.06),
 ('2016-08-25', 0.11),
 ('2016-08-26', 0.0),
 ('2016-08-26', 0.03),
 ('2016-08-26', 0.02),
 ('2016-08-26', 0.04),
 ('2016-08-26', 0.0),
 ('2016-08-26', 0.01),
 ('2016-08-27', 0.0),
 ('2016-08-27', 0.18),
 ('2016-08-27', 0.02),
 ('2016-08-27', 0.0),
 ('2016-08-27', 0.12),
 ('2016-08-27', None),
 ('2016-08-28', 0.01),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.6),
 ('2016-08-28', 2.07),
 ('2016-08-29', 0.0),
 ('2016-08-29', 0.17),
 ('2016-08-29', 0.04),
 ('2016-08-29', None),
 ('2016-08-29', 0.0),
 ('2016-08-29', 0.35),
 ('2016-08-29', 0.9),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.02),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.0),
 (

In [115]:
#Save LastYear_PrcpData as a Pandas DataFrame to view proper linking 
LastYear_PrcpData_df = pd.DataFrame(LastYear_PrcpData)
LastYear_PrcpData_df.head()


,date,prcp
0,2016-08-24,0.08
1,2016-08-24,2.15
2,2016-08-24,2.28
3,2016-08-24,NaN
4,2016-08-24,1.22


In [117]:
# Set the index to the date column

LastYear_PrcpData_df = LastYear_PrcpData_df.set_index("date")
LastYear_PrcpData_df.head()


,prcp
date,
2016-08-24,0.08
2016-08-24,2.15
2016-08-24,2.28
2016-08-24,NaN
2016-08-24,1.22
